In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [36]:
# Step 1. Load the datasets from uploaded files (I downloaded the csv files and placed them in the folder book_knn within this project)

books_filename =  "https://raw.githubusercontent.com/ashwanidv100/Recommendation-System---Book-Crossing-Dataset/master/BX-CSV-Dump/BX-Books.csv"
users_filename =  "https://raw.githubusercontent.com/ashwanidv100/Recommendation-System---Book-Crossing-Dataset/master/BX-CSV-Dump/BX-Users.csv"
ratings_filename =  "https://raw.githubusercontent.com/ashwanidv100/Recommendation-System---Book-Crossing-Dataset/master/BX-CSV-Dump/BX-Book-Ratings.csv"

# Step 1: Load Data from URLs /  import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding="ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'}
)

df_ratings = pd.read_csv(
    ratings_filename,
    encoding="ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'}
)     


In [37]:
# Step 2: Create Dictionary to Map ISBN to Book Title
isbn_to_title = df_books.set_index('isbn')['title'].to_dict()

# Step 3: Filter Data
# Lower rating thresholds temporarily for debugging
user_threshold = 100  # Temporary for debugging
book_threshold = 50   # Temporary for debugging

# Filter users with at least user_threshold ratings
user_counts = df_ratings['user'].value_counts()
df_ratings = df_ratings[df_ratings['user'].isin(user_counts[user_counts >= user_threshold].index)]

# Filter books with at least book_threshold ratings
book_counts = df_ratings['isbn'].value_counts()
df_ratings = df_ratings[df_ratings['isbn'].isin(book_counts[book_counts >= book_threshold].index)]

# Step 4: Replace ISBN with Book Titles
# Use the dictionary to map ISBNs to Book Titles
df_ratings['title'] = df_ratings['isbn'].map(isbn_to_title)

# Drop rows where ISBNs didn't map to any Book-Title (in case of missing data)
df_ratings = df_ratings.dropna(subset=['title'])

# Step 5: Manually Create User-Item Matrix
# Get the list of unique books and users
unique_books = list(df_ratings['title'].unique())
unique_users = list(df_ratings['user'].unique())

# Create a dictionary to store the matrix
user_item_matrix = {}

# Initialize the matrix with zeros
for book in unique_books:
    user_item_matrix[book] = [0] * len(unique_users)

# Fill the matrix with ratings
for _, row in df_ratings.iterrows():
    book = row['title']
    user = row['user']
    rating = row['rating']
    user_index = unique_users.index(user)
    user_item_matrix[book][user_index] = rating

# Convert the dictionary to a list of lists (rows: books, columns: users)
book_user_matrix = np.array(list(user_item_matrix.values()))

# Debugging Step: List all available titles
print("\nList of all book titles:\n", unique_books)

# Step 6: Advanced Normalization
# Subtract mean and divide by standard deviation for each user's ratings
normalized_matrix = []
for row in book_user_matrix:
    mean = np.mean(row)
    std = np.std(row)
    normalized_row = (row - mean) / std if std != 0 else row - mean
    normalized_matrix.append(normalized_row)
normalized_matrix = np.array(normalized_matrix)




List of all book titles:
 ['Politically Correct Bedtime Stories: Modern Tales for Our Life and Times', 'The Lovely Bones: A Novel', 'The Da Vinci Code', 'The Joy Luck Club', 'Wild Animus', 'The Poisonwood Bible: A Novel', 'Bel Canto: A Novel', 'One for the Money (Stephanie Plum Novels (Paperback))', 'The Tao of Pooh', 'Girl in Hyacinth Blue', 'Chocolat', 'The Secret Life of Bees', 'Three To Get Deadly : A Stephanie Plum Novel (A Stephanie Plum Novel)', "Full Tilt (Janet Evanovich's Full Series)", 'Lucky : A Memoir', 'The Dogs of Babel (Today Show Book Club #12)', "White Oleander : A Novel (Oprah's Book Club)", 'White Oleander : A Novel', 'The Jester', 'Me Talk Pretty One Day', 'The Hobbit : The Enchanting Prelude to The Lord of the Rings', 'A Prayer for Owen Meany', 'Silent Witness', 'The Murder Book', "The Dress Lodger (Ballantine Reader's Circle)", 'Stanislaski Sisters', 'Middlesex: A Novel', "The Dive From Clausen's Pier : A Novel (Vintage Contemporaries (Paperback))", 'The Indian 

In [38]:
# Step 7: Build the KNN Model using Correlation Similarity
model = NearestNeighbors(metric='correlation', algorithm='brute')
model.fit(normalized_matrix)

# Step 8: Implement the get_recommends() Function
def get_recommends(book = ""):
    # Check if the book is in the dataset
    if book not in unique_books:
        # Suggest similar titles if the exact match isn't found
        print(f"\nBook '{book}' not found. Searching for similar titles...\n")
        similar_titles = [title for title in unique_books if book.lower() in title.lower()]
        if similar_titles:
            print(f"Did you mean one of these?\n{similar_titles}")
        else:
            print("No similar titles found.")
        return f"Book '{book}' not found in dataset."
    
    # Get the index of the book
    book_idx = unique_books.index(book)
    
    # Find the 5 nearest neighbors
    distances, indices = model.kneighbors(normalized_matrix[book_idx].reshape(1, -1), n_neighbors=6)
    
    # Create the list of recommendations
    recommendations = []
    for i in range(1, len(distances.flatten())):
        similar_book = unique_books[indices.flatten()[i]]
        recommendations.append([similar_book, distances.flatten()[i]])
    
    # Sort recommendations by distance and title for consistent order
    recommendations.sort(key=lambda x: (x[1], x[0]))
    
    # Return the result in the required format
    return [book, recommendations]

    

# Test the function with the provided title
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)
    

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Weight of Water', 0.7991861366366018], ['The Lovely Bones: A Novel', 0.8166724332106098], ['I Know This Much Is True', 0.8395961694586211], ['The Poisonwood Bible: A Novel', 0.8473086007850986], ['Icy Sparks', 0.8491870278144447]]]


In [39]:
# Step 7: Test the Function
def test_book_recommendation():
    test_pass = True
    recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
    if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
        test_pass = False
    recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'I Know This Much Is True', 'The Lovely Bones: A Novel']
    recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
    for i in range(2): 
        if recommends[1][i][0] not in recommended_books:
            test_pass = False
        if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
            test_pass = False
    if test_pass:
        print("You passed the challenge! 🎉🎉🎉🎉🎉")
    else:
        print("You haven't passed yet. Keep trying!")

test_book_recommendation()

You passed the challenge! 🎉🎉🎉🎉🎉


In [ ]:
books = get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")
print(books)     # Ah, close enough

['The Queen of the Damned (Vampire Chronicles (Paperback))', [['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.48722937743080696], ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5419872784602991], ['Interview with the Vampire', 0.736967078328985], ['Lasher: Lives of the Mayfair Witches (Lives of the Mayfair Witches)', 0.812579803450494], ['The Witching Hour (Lives of the Mayfair Witches)', 0.8174130829796965]]]
